## Replication of electricity price forecasting

### Days of the week/ Holidays

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
import holidays

In [3]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
y = df['price']

In [ ]:
X = df['load'], df['weather/temp/snow'], df['coal_price'], df['dow'], df['holidays'], df['month']

In [ ]:
model = sm.OLS(y,X).fit()
model.summary()

transform_to_windows
converts the data from row data into windowed rows where each row is a day with 24 columns representing each hour of the day.

In [11]:
from electricity_price_predictor.data import fetch_data

In [12]:
data = fetch_data('../raw_data/price')

In [13]:
data.dtypes

time     datetime64[ns]
price            object
dtype: object

In [ ]:
# df = pd.read_csv('../raw_data/final_load.csv')

In [14]:
data = data.set_index('time')

In [15]:
data.index.year

Int64Index([2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015, 2015,
            ...
            2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020, 2020],
           dtype='int64', name='time', length=51702)

In [ ]:
# load = get_load()

In [ ]:
# load.index

In [7]:
data.head()

,price
time,
2015-01-01 00:00:00,25.02
2015-01-01 01:00:00,18.29
2015-01-01 02:00:00,16.04
2015-01-01 03:00:00,14.6
2015-01-01 04:00:00,14.95


In [ ]:
# def get_persistence_dataset(path='./data/cleaned_data/energy_loads_2015_2019.csv', index='time', start='2015', stop='2018', shift=0):
#     #load the preprocessed data
#     data = pd.read_csv('./data/cleaned_data/energy_loads_2015_2019.csv', parse_dates=True, index_col=index)

#     #use features preprocessing library to transform data into day and hour slice format.
#     data = transform_to_windows(data)

#     #rename the columns
#     data = rename_cols(data, shift=shift)

#     #standardize the data from 2015-2018
#     data = data[start:stop]

#     return data

## Previous day hour by hour

The previous day hour by hour model will use the energy loads/price from the previous day to forecast the next days' load/price on an hour by hour basis

In [ ]:
def day_hbh(history, days=1):
    """
    History is a dataframe with index as days, and columns hours in the day. 
    
    """
    #retrns the last week in the history data set as the forecast for the next week.
    return history.iloc[-days,:]

In [ ]:
from electricity_price_predictor.data_2 import get_shifted_price, get_load

In [ ]:
prices = get_shifted_price()

In [ ]:
load = get_load()

## Feature engineering

In [ ]:
# Generate 'hour', 'weekday' and 'month' features

for i in range(len(df_final)):
    position = df_final.index[i]
    hour = position.hour
    weekday = position.weekday()
    month = position.month
    df_final.loc[position, 'hour'] = hour
    df_final.loc[position, 'weekday'] = weekday
    df_final.loc[position, 'month'] = month

In [ ]:
# Generate 'business hour' feature

for i in range(len(df_final)):
    position = df_final.index[i]
    hour = position.hour
    if ((hour > 8 and hour < 14) or (hour > 16 and hour < 21)):
        df_final.loc[position, 'business hour'] = 2
    elif (hour >= 14 and hour <= 16):
        df_final.loc[position, 'business hour'] = 1
    else:
        df_final.loc[position, 'business hour'] = 0

In [ ]:
# Generate 'weekend' feature

for i in range(len(df_final)):
    position = df_final.index[i]
    weekday = position.weekday()
    if (weekday == 6):
        df_final.loc[position, 'weekday'] = 2
    elif (weekday == 5):
        df_final.loc[position, 'weekday'] = 1
    else:
        df_final.loc[position, 'weekday'] = 0

In [ ]:
# Generate 'temp_range' for each city

cities = ['Barcelona', 'Bilbao', 'Madrid', 'Seville', 'Valencia']

for i in range(len(df_final)):
    position = df_final.index[i]
    for city in cities:
        temp_max = df_final.loc[position, 'temp_max_{}'.format(city)]
        temp_min = df_final.loc[position, 'temp_min_{}'.format(city)]
        df_final.loc[position, 'temp_range_{}'.format(city)] = abs(temp_max - temp_min)

In [ ]:
# Calculate the weight of every city

total_pop = 6155116 + 5179243 + 1645342 + 1305342 + 987000

weight_Madrid = 6155116 / total_pop
weight_Barcelona = 5179243 / total_pop
weight_Valencia = 1645342 / total_pop
weight_Seville = 1305342 / total_pop
weight_Bilbao = 987000 / total_pop

In [ ]:
cities_weights = {'Madrid': weight_Madrid, 
                  'Barcelona': weight_Barcelona,
                  'Valencia': weight_Valencia,
                  'Seville': weight_Seville,
                  'Bilbao': weight_Bilbao}

In [ ]:
# Generate 'temp_weighted' feature

for i in range(len(df_final)):
    position = df_final.index[i]
    temp_weighted = 0
    for city in cities:
        temp = df_final.loc[position, 'temp_{}'.format(city)]
        temp_weighted += temp * cities_weights.get('{}'.format(city))
    df_final.loc[position, 'temp_weighted'] = temp_weighted